# Streaming from Multiplex Bronze

In this notebook, you will configure a query to consume and parse raw data from a single topic as it lands in the multiplex bronze table configured in the last lesson. We'll continue refining this query in the following notebooks.

## Learning Objectives
By the end of this lesson, you should be able to:
- Describe how filters are applied to streaming jobs
- Use built-in functions to flatten nested JSON data
- Parse and save binary-encoded strings to native types

Declare database and set all path variables.

In [0]:
%run ../Includes/Classroom-Setup-3.2

## Define a Batch Read

Before building our streams, we'll start with a static view of our data. Working with static data can be easier during interactive development as no streams will be triggered. 

Because we're working with Delta Lake as our source, we'll still get the most up-to-date version of our table each time we execute a query.

If you're working with SQL, you can just directly query the **`bronze`** table registered in the previous lesson. 

Python and Scala users can easily create a Dataframe from a registered table.

In [0]:
batch_df = spark.table("bronze")
display(batch_df)

Delta Lake stores our schema information. 

Let's print it out, just to recall it's structure:

In [0]:
%sql
DESCRIBE bronze

Preview your data.

In [0]:
%sql
SELECT *
FROM bronze
LIMIT 20

There are multiple topics being ingested. So, we'll need to define logic for each of these topics separately.

In [0]:
%sql
SELECT DISTINCT(topic)
FROM bronze

We'll cast our binary fields as strings, as this will allow us to manually review their contents.

In [0]:
%sql
SELECT cast(key AS STRING), cast(value AS STRING)
FROM bronze
LIMIT 20

## Parse Heart Rate Recordings

Let's start by defining logic to parse our heart rate recordings. We'll write this logic against our static data. Note that there are some <a href="https://spark.apache.org/docs/latest/structured-streaming-programming-guide.html#unsupported-operations" target="_blank">unsupported operations</a> in Structured Streaming, so we may need to refactor some of our logic if we don't build our current queries with these limitations in mind.

Together, we'll iteratively develop a single query that parses our **`bpm`** topic to the following schema.

| field | type |
| --- | --- |
| device_id | LONG | 
| time | TIMESTAMP | 
| heartrate | DOUBLE |

We'll be creating the table **`heartrate_silver`** in our architectural diagram.

<img src="https://files.training.databricks.com/images/ade/ADE_arch_heartrate_silver.png" width="60%" />

In [0]:
%sql
SELECT v.*
FROM (
  SELECT from_json(cast(value AS STRING), "device_id LONG, time TIMESTAMP, heartrate DOUBLE") v
  FROM bronze
  WHERE topic = "bpm")

## Convert Logic for Streaming Read

We can define a streaming read directly against our Delta table. Note that most configuration for streaming queries is done on write rather than read, so here we see little change to our above logic.

The cell below shows how to convert a static table into a streaming temp view (if you wish to write streaming queries with Spark SQL).

In [0]:
(spark.readStream
      .table("bronze")
      .createOrReplaceTempView("TEMP_bronze"))

Updating our above query to refer to this temp view gives us a streaming result.

In [0]:
%sql
SELECT v.*
FROM (
  SELECT from_json(cast(value AS STRING), "device_id LONG, time TIMESTAMP, heartrate DOUBLE") v
  FROM TEMP_bronze
  WHERE topic = "bpm")

Note that anytime a streaming read is displayed to a notebook, a streaming job will begin and if allowed to run forever this will prevent the cluster from auto-terminating.  You can stop the stream clicking the "Cancel" link in the cell above, clicking "Stop Execution" at the top of the notebook, or running the code below.

Stop the streaming display above before continuing.

In [0]:
for stream in spark.streams.active:
    stream.stop()
    stream.awaitTermination()

To persist results to disk, a streaming write will need to be performed using Python.  We can switch from SQL to Python by using a temporary view as an intermediary to capture the query we want to apply.

In [0]:
%sql
CREATE OR REPLACE TEMPORARY VIEW TEMP_SILVER AS
  SELECT v.*
  FROM (
    SELECT from_json(cast(value AS STRING), "device_id LONG, time TIMESTAMP, heartrate DOUBLE") v
    FROM TEMP_bronze
    WHERE topic = "bpm")

Read from the streaming **`TEMP_SILVER`** temporary view and write to the **`heart_rate_silver`** delta table.

Using the **`trigger(availableNow=True)`** option will process all records (in multiple batches if needed) until no more data is available and then stop the stream.

In [0]:
query = (spark.table("TEMP_SILVER").writeStream
               .option("checkpointLocation", f"{DA.paths.checkpoints}/heart_rate")
               .option("path", f"{DA.paths.user_db}/heart_rate_silver.delta")
               .trigger(availableNow=True)
               .table("heart_rate_silver"))

query.awaitTermination()

Alternatively, instead of using SQL, the entire job can be expressed using Python Dataframes API.  The cell below has this logic refactored to Python.

In [0]:
from pyspark.sql import functions as F

json_schema = "device_id LONG, time TIMESTAMP, heartrate DOUBLE"

(spark
   .readStream.table("bronze")
   .filter("topic = 'bpm'")
   .select(F.from_json(F.col("value").cast("string"), json_schema).alias("v"))
   .select("v.*")
   .writeStream
       .option("checkpointLocation", f"{DA.paths.checkpoints}/heart_rate")
       .option("path", f"{DA.paths.user_db}/heart_rate_silver.delta")
       .trigger(availableNow=True)
       .table("heart_rate_silver"))

query.awaitTermination()

<img src="https://files.training.databricks.com/images/icon_warn_32.png"> Before continuing, make sure you cancel any streams. The **`Run All`** button at the top of the screen will say **`Stop Execution`** if you have a stream still running.  Or run the code below to stop all streaming currently running on this cluster.

In [0]:
for stream in spark.streams.active:
    stream.stop()
    stream.awaitTermination()

## Silver Table Motivations

In addition to parsing records and flattening and changing our schema, we should also check the quality of our data before writing to our silver tables.

In the following notebooks, we'll review various quality checks.

Run the following cell to delete the tables and files associated with this lesson.

In [0]:
DA.cleanup()